In [42]:
# import libraries

import os
import requests
import pandas as pd
import json
from pathlib import Path
import numpy as np
import datetime
import dateutil.parser
import urllib.request
from dotenv import load_dotenv

#data viz

import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
import altair as alt



%matplotlib inline

# import for interactive panels

import panel as pn
from panel.interact import interact
from panel import widgets
#pn.extension('plotly')
pn.extension()


In [43]:
import hvplot.pandas

In [44]:
# Read the API keys
load_dotenv()

True

In [45]:
# Create latest up to date dataset for each position reviewed:
# A historical dataset is created by a separate script that pulls, massages the data and creates a csv.
# This data is then pulled into a df and then updated with latest market information using appropriate API
# get request. This way historical data doesnt need to be retrieved with repetitive API gets, but the data 
# can be kept up to date with latest info.

# #each should return, cleaned properly and format
#def BondAgg_latest_data
#
#date adj close price
#
#def BTCUSD_latest_data
#
#def XAUUSD_latest_data

## Fetch ART Data ##
art_file_path = Path("../data_imports/Artprice_Indexes_Quarterly_data_Base100_January1998.csv")
art_index = pd.read_csv(art_file_path, encoding='utf-8', parse_dates=True, infer_datetime_format=True)
art_index.rename(columns=({"Unnamed: 0" : "Date"}), inplace=True)
art_index.dropna()
art_index.set_index("Date", drop=True, inplace=True)
#art_index.tail()

In [46]:
## Fetch SP500 Data ##
ms_api_key = os.getenv("MARKETSTACK_API_KEY")
ms_api_key_string = "&access_key=" + ms_api_key 
print(type(ms_api_key))
ms_request_url = "https://api.marketstack.com/v1/eod?symbols=GSPC.INDX"
ms_request_url = ms_request_url + ms_api_key_string + "&limit=3000" 
# Execute get request with API key
ms_response_data = requests.get(ms_request_url)
ms_data = ms_response_data.json()
#print(json.dumps(data, indent=4))
sp_list = ms_data["data"]
# Pull json outout into dataframe and update the adjusted closing prices into columns
column_list = ['Date', 'Position', 'AdjClose']
sp500_df = pd.DataFrame(columns=column_list)
for dict in sp_list: 
    for list in dict: 
        if list == "date":
            temp_date = str(dateutil.parser.parse(dict[list]).date())
            #print(temp_date)
        elif list == "adj_close":
            temp_close = dict[list] 
            
    new_row = {'Date':temp_date, 'Position':'SP500', 'AdjClose':temp_close}
    sp500_df = sp500_df.append(new_row, ignore_index=True)                 
    
sp500_df["Date"] = pd.to_datetime(sp500_df["Date"])
sp500_df.set_index("Date", drop=True, inplace=True)
#sp500_df.rename(columns = {"sp500_adj_closing" : "Price"}, inplace=True)

sp500_df.head()


<class 'str'>


,Position,AdjClose
Date,,
2020-11-04,SP500,3443.4399
2020-11-03,SP500,3369.1599
2020-11-02,SP500,3310.2400
2020-10-30,SP500,3269.9600
2020-10-29,SP500,3310.1101


In [47]:
## Fetch crypto Data ##
api_key = os.getenv("NOMICS_API_KEY")
print(type(api_key))
api_key_string = "key=" + api_key
request_url = "https://api.coindesk.com/v1/bpi/historical/close.json?start=2012-01-01&end=2020-10-05"
# Execute get request
response_data = requests.get(request_url)
data = response_data.json()
#print(json.dumps(data, indent=4))
bpi_list = data["bpi"]
#print(json.dumps(bpi_list, indent=4))
bpi_df = pd.DataFrame.from_dict(bpi_list, orient='index',
                       columns=['bpi_closing'])
#bpi_df.columns = ['bpi_closing']
#bpi_df.set_index(bpi_df['date'],inplace=True)
bpi_df.head()

<class 'str'>


,bpi_closing
2012-01-01,5.2677
2012-01-02,5.2168
2012-01-03,4.8808
2012-01-04,5.5738
2012-01-05,6.9476


In [48]:
# first select art index of choice and scale up price by 100 & add position name column
gblart_index_col_list = ['Position','AdjClose']
gblart_index_df = pd.DataFrame(columns=gblart_index_col_list)
gblart_index_df['AdjClose'] = (art_index.loc[:,'Global Index (USD)'])*100
gblart_index_df['Position'] = 'GlobalArtIndex'
gblart_index_df.head()


,Position,AdjClose
Date,,
1998-01-01,GlobalArtIndex,10000
1998-04-01,GlobalArtIndex,10700
1998-07-01,GlobalArtIndex,10600
1998-10-01,GlobalArtIndex,9600
1999-01-01,GlobalArtIndex,10400


In [49]:
####################################################################
# EVERYTHING ABOVE WILL BE REVISED ONCE DATASETS FUNCTIONS CREATED
# BELOW STARTS THE FINAL VERSION OF ANALYTICS AND VISUALIZATIONS
####################################################################
# 
# THIS CELL TEMPORARILY COMMENTED OUT TO QUICK HACK SOMETHING FOR KRIS PLOTS
#
####################################################################

# Define positions (indexes, algorithmitic portfolios, assets to evaluate
# Define currencies to review performance against
# Define evaluation calculations to perform
# All are manually coded in for first MVP
#eval_pos_df_list = [
#                #'FAANG',
#                #'BondAgg',
#                #'Resi_RE',
#                #'GlobalArtIndex',
#                sp500_df
#                ]
#currency_df_list = [
#                USD,
#                BTCUSD,
#                XAUUSD
#                ]
#calc_list = ['price','returns_daily','returns_cum','std']
#
#
#calc_dict = {
#            pos : [pos_price_USD_df,pos_price_]
#}


In [50]:
####################################################################
# 
# THIS CELL TEMPORARILY COMMENTED OUT TO QUICK HACK SOMETHING FOR KRIS PLOTS
#
####################################################################
#def sanitize_dataset(dataset_df):
#    """Insure data sorted ascending"""
#    dataset_df = dataset_df.sort_index()
#    print(dataset_df.head())
#    return(dataset_df)
#
## Sweep each position,for each currency, for each calculation
#for pos in eval_pos_list:
#    # first do finalize sanitation step on incoming position dataset
#    print(type(pos))
#    sanitize_dataset(pos)
#    for currency in currency_df_list:
#        if (currency == USD):
# 

In [51]:
# Create long-form USD price df for all pos
price_frames = [
                #'FAANG',
                #'BondAgg',
                #'Resi_RE',
                gblart_index_df,
                sp500_df
                ]                    

lcombo_USD_df = pd.concat(price_frames)
#lcombo_USD_df

lcombo_USD_df['Returns'] = lcombo_USD_df['AdjClose'].pct_change()
#lcombo_USD_df

#lcombo_USD_df['STD'] = lcombo_USD_df['AdjClose'].groupby('Position').std()
lcombo_USD_df['STD'] = lcombo_USD_df['AdjClose'].std()
lcombo_USD_df   

,Position,AdjClose,Returns,STD
Date,,,,
1998-01-01,GlobalArtIndex,10000.0000,NaN,2668.647581
1998-04-01,GlobalArtIndex,10700.0000,0.070000,2668.647581
1998-07-01,GlobalArtIndex,10600.0000,-0.009346,2668.647581
1998-10-01,GlobalArtIndex,9600.0000,-0.094340,2668.647581
1999-01-01,GlobalArtIndex,10400.0000,0.083333,2668.647581
1999-04-01,GlobalArtIndex,10300.0000,-0.009615,2668.647581
1999-07-01,GlobalArtIndex,10400.0000,0.009709,2668.647581
1999-10-01,GlobalArtIndex,10400.0000,0.000000,2668.647581
2000-01-01,GlobalArtIndex,11300.0000,0.086538,2668.647581


In [58]:
group = lcombo_USD_df.groupby(("Position"))

source = ColumnDataSource(data=group)

group.head(100)


,Position,AdjClose,Returns,STD
Date,,,,
1998-01-01,GlobalArtIndex,10000.0000,NaN,2668.647581
1998-04-01,GlobalArtIndex,10700.0000,0.070000,2668.647581
1998-07-01,GlobalArtIndex,10600.0000,-0.009346,2668.647581
1998-10-01,GlobalArtIndex,9600.0000,-0.094340,2668.647581
1999-01-01,GlobalArtIndex,10400.0000,0.083333,2668.647581
1999-04-01,GlobalArtIndex,10300.0000,-0.009615,2668.647581
1999-07-01,GlobalArtIndex,10400.0000,0.009709,2668.647581
1999-10-01,GlobalArtIndex,10400.0000,0.000000,2668.647581
2000-01-01,GlobalArtIndex,11300.0000,0.086538,2668.647581


In [65]:

plot = figure(plot_width=700, plot_height=400)

plot.line([lcombo_USD_df], x="Date", source=source)

plot

RuntimeError: 

Expected x to reference fields in the supplied data source.

When a 'source' argument is passed to a glyph method, values that are sequences
(like lists or arrays) must come from references to data columns in the source.

For instance, as an example:

    source = ColumnDataSource(data=dict(x=a_list, y=an_array))

    p.circle(x='x', y='y', source=source, ...) # pass column names and a source

Alternatively, *all* data sequences may be provided as literals as long as a
source is *not* provided:

    p.circle(x=a_list, y=an_array, ...)  # pass actual sequences and no source



In [60]:
# HENRY SCRATCHPAD

#for currency in currency_list:
#    for pos in eval_pos_list:
#        # grab pos dataset and insure arranged ascending
#        calc_df = {pos}
       
# calculate for each pos in each currency then concat at end as combo_calc_currency
#benchmk_pos_list = ['SP500','GlobalArtIndex']
#
#for pos in benchmk_pos_list:
 #   print(f"the ticker is {pos}")

#test_price_df = pd.DataFrame(columns=benchmk_pos_list)
#test_price_df = sp500_df
#
#test_price_df =  test_price_df.merge(
#                                    gblart_index_df, 
#                                    how='left',
#                                    left_index=True,
#                                    right_index=True
#                                    )
#
#test_returns_df = test_price_df.pct_change() #
#test_std_df = test_price_df.std()
#test_std_df
#
#

#sp500_daily_returns = sp500_df.pct_change().dropna()
#sp500_daily_returns.head()

In [139]:
#gblart_returns_df = pd.DataFrame(columns=['GlobalArtIndex'])
#gblart_returns_df['GlobalArtIndex'] = art_index.loc[:,'Global Index (USD)'].pct_change().dropna()
#test_combo_plot = gblart_returns_df.hvplot.line(title="test combo Returns", ylabel="Daily Returns", frame_width=900, frame_height=450, rot=90)
#test_combo_plot
#gblart_index_df = pd.DataFrame(columns=['GlobalArtIndex'])



In [140]:
#returns_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)
#cum_returns_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)
#std_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)
#sharpe_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)

#returns_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)
#cum_returns_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)
#std_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)
#sharpe_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)

#returns_eval_10yr_df['GlobalArtIndex'] = art_index.loc[:,'Global Index (USD)'].pct_change().dropna()
#returns_eval_10yr_df.head()

# test merge trying zero fill NaN locations to help with line plot below -- spoiler alert -- doesnt work
#test_combo_df = sp500_daily_returns.merge(gblart_returns_df,how='left',left_index=True,right_index=True).fillna(0)
# long head here to verify that its joining where there's valid art data
#test_combo_df.head(100)


In [141]:
#test_combo_plot = test_combo_df.hvplot.line(title="test combo ", ylabel="Returns", frame_width=900, frame_height=450, rot=90)
#test_combo_plot

In [8]:
####################################################################
# THIS CELL IS START OF VISUALIZATION FUNCTIONS 
#
####################################################################
# Define Panel Visualization Functions
# Number of plot functions right now = 5 calcs/position * 2timeframes * 6 positions = 60!!!!


#def 
#art_index_std_bar = px.bar(art_index_std, title="Art Price STD 1998-2020")
#art_index_std_bar

In [2]:
# Create Panel Dashboard

